In [ ]:
from langgraph.graph import StateGraph , START , END
from typing import TypedDict , Literal , Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage , HumanMessage
import os
from pydantic import BaseModel

In [2]:
load_dotenv()

google_apikey  = os.getenv("GEMINI_API_KEY")

generator_llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-pro",
    api_key = google_apikey
)


evaluation_llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-pro",
    api_key = google_apikey
)


optimizer_llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-pro",
    api_key = google_apikey
)

In [ ]:
class TweetState(TypedDict):
    topic:str
    generate_tweet:str
    evaluation : Literal['Approved' , "needs_improvement"]
    feedback :str
    iteration : int
    max_iteratiom : int


In [ ]:
def generate_tweet(state:TweetState):
        messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]
    # send generator_llm
        response = generator_llm.invoke(messages).content

    # return response
        return {'tweet': response, 'tweet_history': [response]}




In [ ]:
def evaluate_tweet(state: TweetState):

    # prompt
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
]

    response = evaluation_llm.invoke(messages)

    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

In [ ]:
graph = StateGraph(TweetState)


graph.add_node("generate_tweet" , generate_tweet)
graph.add_node("evaluate_tweet" , evaluate_tweet)
graph.add_node('optimize_tweet' , optimize_tweet)

